In [1]:
import json
import re
import numpy as np 
import pandas as pd
from time import sleep
import es_dep_news_trf
import pt_core_news_lg

In [2]:
import importlib.util as imp
import sys

spec = imp.spec_from_file_location(
    'twitter_connection', 
    '../twitter-connection/__init__.py')
twit = imp.module_from_spec(spec)
sys.modules[spec.name] = twit
spec.loader.exec_module(twit)

from twitter_connection import connection as tc
from twitter_connection import response as tr

In [3]:
'''
File path to the bearer token. Requires a prefix to identify the
  token, which is just 'PERSONAL $BEARER_TOKEN$' by default -- 
  can be specified upon initialization of TwitterConnection
'''
cred_path = r'../twitter-connection/credentials.txt'

In [4]:
with open('../extraction/verb-stem-clean.txt') as f:
    verb_stem = json.load(f)

In [5]:
es_query_cond = 'lang:es has:geo -is:retweet -has:links '
pt_query_cond = 'lang:pt has:geo -is:retweet -has:links '
fields_tweet = 'tweet.fields=lang,geo,created_at,public_metrics,referenced_tweets'
fields_expan = 'expansions=author_id,geo.place_id,entities.mentions.username'
fields_user = 'user.fields=created_at,location,public_metrics'
fields_place = 'place.fields=country'

In [6]:
es_conn = tc.TwitterConnection(
    is_archive=True,
    cred_prefix='PROF')

pt_conn = tc.TwitterConnection(
    is_archive=True,
    cred_prefix='PROF') 

In [7]:
es_conn.set_query(conditions=es_query_cond)
es_conn.set_fields(tweet=fields_tweet, 
                      expansions=fields_expan, 
                      user=fields_user,
                      place=fields_place)

pt_conn.set_query(conditions=pt_query_cond)
pt_conn.set_fields(tweet=fields_tweet, 
                      expansions=fields_expan, 
                      user=fields_user,
                      place=fields_place)

In [14]:
# s = None
# for vs in verb_stem.items():
#     s = '(' + vs[1] + ')'
#     break

s = '(' + verb_stem['decir'] + ')'

In [15]:
es_conn.connect(s, is_next=True)
sleep(1)
pt_conn.connect(s, is_next=True)

URL: https://api.twitter.com/2/tweets/search/all?query=(dig OR dec OR dij OR dir OR dic) lang:es has:geo -is:retweet -has:links &max_results=100&next_token=b26v89c19zqg8o3fpdgapree2rz4ydktm28uh2trjosjh&tweet.fields=lang,geo,created_at,public_metrics,referenced_tweets&expansions=author_id,geo.place_id,entities.mentions.username&user.fields=created_at,location,public_metrics&place.fields=country
URL: https://api.twitter.com/2/tweets/search/all?query=(dig OR dec OR dij OR dir OR dic) lang:pt has:geo -is:retweet -has:links &max_results=100&next_token=b26v89c19zqg8o3fpdgapree2rz4yzh1xg1zif4n7gy9p&tweet.fields=lang,geo,created_at,public_metrics,referenced_tweets&expansions=author_id,geo.place_id,entities.mentions.username&user.fields=created_at,location,public_metrics&place.fields=country


True

In [16]:
tr.save_json(f'es-{s}-test-data.txt', es_conn.response)
tr.save_json(f'pt-{s}-test-data.txt', pt_conn.response)

In [17]:
es = tr.Response(tr.retrieve(f'es-{s}-test-data.txt'))
pt = tr.Response(tr.retrieve(f'pt-{s}-test-data.txt'))

In [18]:
display(es.schema['data'].head(3))
display(pt.schema['data'].head(3))

,author_id,id,created_at,lang,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,entities.mentions,geo.place_id,referenced_tweets
0,470689531,1409900882016276480,2021-06-29T15:45:39.000Z,es,"Rodrigo Perpétuo, dir de ICLEI para América,en...",0,0,0,0,"[{'start': 259, 'end': 272, 'username': 'Terri...",00698bbe577bad5b,NaN
1,114938653,1409895036121194502,2021-06-29T15:22:25.000Z,es,La Caja Dig está bastante bien salvo el canal ...,0,1,0,0,NaN,008eb02a542ac3d4,"[{'type': 'replied_to', 'id': '140989503379755..."
2,114938653,1409895033797554178,2021-06-29T15:22:25.000Z,es,@TuMundoInter @InterCliente . Cliente No 25000...,0,1,0,0,"[{'start': 0, 'end': 13, 'username': 'TuMundoI...",008eb02a542ac3d4,NaN


,author_id,text,referenced_tweets,created_at,id,lang,geo.place_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,entities.mentions
0,302551162,@josepalay De quina manera hem de dir prou?,"[{'type': 'replied_to', 'id': '140984123013398...",2021-06-29T15:35:52.000Z,1409898421352345600,pt,4249ed07bb463097,0,0,0,0,"[{'start': 0, 'end': 10, 'username': 'josepala..."
1,1067005281022951424,"Mds eu amo malhar, essa dir muscular pós trein...","[{'type': 'replied_to', 'id': '140970168069471...",2021-06-29T02:34:06.000Z,1409701682766761984,pt,75aa9575e5c66006,0,0,0,0,NaN
2,1238851765820366851,@Maximin17035909 @Consu72907765 Bona tarda...P...,"[{'type': 'replied_to', 'id': '140950035616224...",2021-06-28T16:50:32.000Z,1409554820357361665,pt,0c3b95d73e721bd9,0,1,1,0,"[{'start': 0, 'end': 16, 'username': 'Maximin1..."


In [19]:
from unidecode import unidecode

In [20]:
nlp_es = es_dep_news_trf.load()
nlp_pt = pt_core_news_lg.load()

In [21]:
def get_pos_tags(tokenized):
    return ' '.join([f'{t.text}-({t.pos_})' for t in tokenized if ((t.pos_!='PUNCT') and (t.pos_!='SPACE'))])

In [22]:
def lemmatize(tokenized):
    return ' '.join([t.lemma_ for t in tokenized])

In [23]:
def analyze(text, lang):
    # Tokenized
    text_nlp = text.loc[:, 'text'].apply(nlp_es if lang=='es' else nlp_pt)
    
    pos = text_nlp.apply(get_pos_tags).rename('pos')
    lemma = text_nlp.apply(lemmatize).rename('lemma')
    
    return pd.concat([text.loc[:, 'id'], pos, lemma], axis=1)

In [24]:
verb = 'decir'
stems = '(dig OR dec OR dij OR dir OR dic)'

In [25]:
response = tr.Response()

In [30]:
es = tr.Response(tr.retrieve(f'es-{s}-test-data.txt'))

In [31]:
# Remove '@...' mentions
es.schema['data'].loc[:, 'text'] = es.schema['data'].loc[:, 'text']\
    .str.replace(r'(@[\w]+ )', '', regex=True)\
    .str.lower()

text_analyzed = analyze(es.schema['data'].loc[:, ['id', 'text']], 'es')

display(text_analyzed.head())

# Entries without desired verb
no_verb = ~(text_analyzed.loc[:, 'lemma'].str.contains(verb))
print(f'Found {no_verb.sum()} without "{verb}"')

display(pd.concat([es.schema['data'].loc[:, 'text'], es.schema['data'].loc[no_verb, :]], axis=1))

es.join(to='data', data=text_analyzed, on='id')

es.schema['data'].drop(es.schema['data'].loc[no_verb, :].index, inplace=True)
response.reset_index()       
    
response.append(es)

response.to_csv('es', verb)

,id,pos,lemma
0,1409900882016276480,rodrigo-(NOUN) perpétuo-(ADJ) dir-(NOUN) de-(A...,"rodrigo perpétuo , dir de iclei para américa ,..."
1,1409895036121194502,la-(DET) caja-(NOUN) dig-(NOUN) está-(AUX) bas...,el caja dig estar bastante bien salvo el canal...
2,1409895033797554178,cliente-(NOUN) no-(ADV) 2500035893-(NUM) la-(D...,. cliente no 2500035893 . el semana pasado yo ...
3,1409866853615288321,hijos-(NOUN),hijo
4,1409850468923330561,,qepd ✝ ️


Found 89 without "decir"


,text,author_id,id,created_at,lang,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,entities.mentions,geo.place_id,referenced_tweets
0,"rodrigo perpétuo, dir de iclei para américa,en...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,la caja dig está bastante bien salvo el canal ...,114938653,1409895036121194502,2021-06-29T15:22:25.000Z,es,la caja dig está bastante bien salvo el canal ...,0.0,1.0,0.0,0.0,NaN,008eb02a542ac3d4,"[{'type': 'replied_to', 'id': '140989503379755..."
2,. cliente no 2500035893. la semana pasada me l...,114938653,1409895033797554178,2021-06-29T15:22:25.000Z,es,. cliente no 2500035893. la semana pasada me l...,0.0,1.0,0.0,0.0,"[{'start': 0, 'end': 13, 'username': 'TuMundoI...",008eb02a542ac3d4,NaN
3,hijos,800871196958998528,1409866853615288321,2021-06-29T13:30:26.000Z,es,hijos,0.0,0.0,0.0,0.0,"[{'start': 0, 'end': 13, 'username': 'Dic_Juri...",3ad512d283f67a11,"[{'type': 'replied_to', 'id': '140986451995068..."
4,qepd✝️,139486415,1409850468923330561,2021-06-29T12:25:20.000Z,es,qepd✝️,0.0,0.0,0.0,0.0,"[{'start': 0, 'end': 9, 'username': 'facesuc1'...",015d19216c749864,"[{'type': 'replied_to', 'id': '140984182899263..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,a dir poco scandaloso …,1372986890,1407608353950478346,2021-06-23T07:55:58.000Z,es,a dir poco scandaloso …,0.0,0.0,1.0,0.0,"[{'start': 0, 'end': 16, 'username': 'matteosa...",7d588036fe12e124,"[{'type': 'replied_to', 'id': '140760666963962..."
96,pollo caducado? \nla justícia res a dir?,4310584995,1407603468701949953,2021-06-23T07:36:33.000Z,es,pollo caducado? \nla justícia res a dir?,0.0,0.0,0.0,0.0,"[{'start': 0, 'end': 12, 'username': 'FonsiLoa...",1a27537478dd8e38,"[{'type': 'replied_to', 'id': '140724628473029..."
97,"ahir vaig dir ""buà qué fuerte, ya es casi navi...",188693013,1407586566323703810,2021-06-23T06:29:23.000Z,es,"ahir vaig dir ""buà qué fuerte, ya es casi navi...",0.0,0.0,1.0,0.0,"[{'start': 0, 'end': 8, 'username': 'VicJota',...",1a27537478dd8e38,"[{'type': 'replied_to', 'id': '140756102524797..."
98,si nos llevamos del gobierno !!10 cepas nueva...,342302070,1407462818362822667,2021-06-22T22:17:39.000Z,es,si nos llevamos del gobierno !!10 cepas nueva...,0.0,1.0,0.0,0.0,"[{'start': 0, 'end': 14, 'username': 'Helenita...",01fcc4a23f17e1ed,"[{'type': 'replied_to', 'id': '140745835445317..."


Before append: 17


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [40]:
%%time
es_t = es_text.apply(unidecode)
pt_t = pt_text.apply(unidecode)

es_bad = is_bad_verb('es', es_t, 'vi OR ve OR ve')
pt_bad = is_bad_verb('pt', pt_t, 'vi OR ve OR ve')

,text,pos,lemma
0,Loq se ve no se pregunta dijo la juanga!...,loq (PRON) se (PRON) ve (VERB) no (ADV) se (PR...,loq él ver no él preguntar decir el juanga ! ...
1,"Barbaro ese Sanjuanero, tenemos que llevarlo a...","barbaro (ADJ) ese (DET) sanjuanero (NOUN) , (P...","barbaro ese sanjuanero , tener que llevar él a..."
2,"Perdon...marido lo ve, no quiero que gane.",perdon (INTJ) ... (PUNCT) marido (NOUN) lo (PR...,"perdon ... marido él ver , no querer que ganar ."
3,no vamos a dar el gusto ve dueno las obras co...,no (ADV) vamos (AUX) a (ADP) dar (VERB) el (DE...,no ir a dar el gusto ver dueno el obra const...
4,"A ese ministro no lo vi recorrer la granja, si...",a (ADP) ese (DET) ministro (NOUN) no (ADV) lo ...,"a ese ministro no él ver recorrer el granja , ..."
...,...,...,...
195,Argentina venia devastada economicamente.\nPor...,argentina (ADJ) venia (VERB) devastada (ADJ) e...,argentino venia devastado economicamente . \n ...
196,"En #Medellin se ve uso de gases lijados, ESMAD...",en (ADP) # (PUNCT) medellin (NOUN) se (PRON) v...,"en # medellin él ver uso de gas lijado , esmad..."
197,"Te iba a enamorar pero vi la foto de tu ex, y ...",te (PRON) iba (AUX) a (ADP) enamorar (VERB) pe...,"tú ir a enamorar pero ver el foto de tu ex , y..."
198,Mir!! Ve el programa completo. Vale la pena.,mir (PROPN) ! (PUNCT) ! (PUNCT) ve (VERB) el (...,mir ! ! ver el programa completo . valer el pe...


,text,pos,lemma
0,Acordei de manha passando mal e fiquei assim o...,acordei (VERB) de (ADP) manha (NOUN) passando ...,acordar de manha passar mal e ficar assim o di...
1,primeira vez que eu te vi tao bom que foi nos ...,primeira (ADP) vez (NOUN) que (SCONJ) eu (PRON...,primeiro vez que eu te vir tao bom que ser o d...
2,"Fui ve um video no Instagram do lou tocando, m...",fui (AUX) ve (VERB) um (DET) video (NOUN) no (...,"ser ve um video o instagram do lou tocar , me ..."
3,Nao vi o nome do senador Marcos Rogerio...\nNa...,nao (ADV) vi (VERB) o (DET) nome (NOUN) do (AD...,nao vir o nome do senador marco rogerio ... \n...
4,"Eu so queria te contar que eu fui la fora, e v...",eu (PRON) so (X) queria (VERB) te (PRON) conta...,"eu so querer te contar que eu ser o ser , e vi..."
...,...,...,...
95,"Eu vi no feed de alguem, mesmo sem querer ver.",eu (PRON) vi (VERB) no (ADP) feed (NOUN) de (A...,"eu vir o feed de alguem , mesmo sem querer v..."
96,O guri nem e tudo aquilo e SE ACHA.... sem fal...,o (DET) guri (NOUN) nem (CCONJ) e (CCONJ) tudo...,o guri nem e tudo aquilo e se achar .... sem f...
97,"Ontem vi um Post ""casar com 22 anos e a mesma ...","ontem (ADV) vi (VERB) um (DET) post (NOUN) "" (...","ontem vir um post "" casar com 22 ano e o mesmo..."
98,"Po paizao , hoje trampo foi foda tiver que man...","po (SCONJ) paizao (VERB) , (PUNCT) hoje (ADV) ...","po paizao , hoje trampo ser foder ter que mand..."


,text,pos,lemma
0,Loq se ve no se pregunta dijo la juanga!...,loq (PRON) se (PRON) ve (VERB) no (ADV) se (PR...,loq él ver no él preguntar decir el juanga ! ...
1,"Barbaro ese Sanjuanero, tenemos que llevarlo a...","barbaro (ADJ) ese (DET) sanjuanero (NOUN) , (P...","barbaro ese sanjuanero , tener que llevar él a..."
2,"Perdon...marido lo ve, no quiero que gane.",perdon (INTJ) ... (PUNCT) marido (NOUN) lo (PR...,"perdon ... marido él ver , no querer que ganar ."
3,no vamos a dar el gusto ve dueno las obras co...,no (ADV) vamos (AUX) a (ADP) dar (VERB) el (DE...,no ir a dar el gusto ver dueno el obra const...
4,"A ese ministro no lo vi recorrer la granja, si...",a (ADP) ese (DET) ministro (NOUN) no (ADV) lo ...,"a ese ministro no él ver recorrer el granja , ..."
...,...,...,...
195,Argentina venia devastada economicamente.\nPor...,argentina (ADJ) venia (VERB) devastada (ADJ) e...,argentino venia devastado economicamente . \n ...
196,"En #Medellin se ve uso de gases lijados, ESMAD...",en (ADP) # (PUNCT) medellin (NOUN) se (PRON) v...,"en # medellin él ver uso de gas lijado , esmad..."
197,"Te iba a enamorar pero vi la foto de tu ex, y ...",te (PRON) iba (AUX) a (ADP) enamorar (VERB) pe...,"tú ir a enamorar pero ver el foto de tu ex , y..."
198,Mir!! Ve el programa completo. Vale la pena.,mir (PROPN) ! (PUNCT) ! (PUNCT) ve (VERB) el (...,mir ! ! ver el programa completo . valer el pe...


,text,pos,lemma
0,Acordei de manha passando mal e fiquei assim o...,acordei (VERB) de (ADP) manha (NOUN) passando ...,acordar de manha passar mal e ficar assim o di...
1,primeira vez que eu te vi tao bom que foi nos ...,primeira (ADP) vez (NOUN) que (SCONJ) eu (PRON...,primeiro vez que eu te vir tao bom que ser o d...
2,"Fui ve um video no Instagram do lou tocando, m...",fui (AUX) ve (VERB) um (DET) video (NOUN) no (...,"ser ve um video o instagram do lou tocar , me ..."
3,Nao vi o nome do senador Marcos Rogerio...\nNa...,nao (ADV) vi (VERB) o (DET) nome (NOUN) do (AD...,nao vir o nome do senador marco rogerio ... \n...
4,"Eu so queria te contar que eu fui la fora, e v...",eu (PRON) so (X) queria (VERB) te (PRON) conta...,"eu so querer te contar que eu ser o ser , e vi..."
...,...,...,...
95,"Eu vi no feed de alguem, mesmo sem querer ver.",eu (PRON) vi (VERB) no (ADP) feed (NOUN) de (A...,"eu vir o feed de alguem , mesmo sem querer v..."
96,O guri nem e tudo aquilo e SE ACHA.... sem fal...,o (DET) guri (NOUN) nem (CCONJ) e (CCONJ) tudo...,o guri nem e tudo aquilo e se achar .... sem f...
97,"Ontem vi um Post ""casar com 22 anos e a mesma ...","ontem (ADV) vi (VERB) um (DET) post (NOUN) "" (...","ontem vir um post "" casar com 22 ano e o mesmo..."
98,"Po paizao , hoje trampo foi foda tiver que man...","po (SCONJ) paizao (VERB) , (PUNCT) hoje (ADV) ...","po paizao , hoje trampo ser foder ter que mand..."


KeyboardInterrupt: 

In [14]:
es_out = pd.concat(
    [es.schema['data'].loc[:, 'text'], es_bad], axis=1)
pt_out = pd.concat(
    [pt.schema['data'].loc[:, 'text'], pt_bad], axis=1)

In [15]:
d.to_csv('es', 'ver')

In [33]:
es_bad.loc[:, 'is_duplicate'].sum()

100

In [34]:
display(es_out.head())
display(pt_out.head())

,text,lemmad,is_duplicate,no_stem,has_verb
0,@cesarsmedina14 Loq se ve no se pregunta dijo ...,loq él ver no él preguntar decir el juanga ! ...,False,False,True
1,@Jlfuentes24 @luisegurrolah Bárbaro ese Sanjua...,"barbaro ese sanjuanero , tener que llevar él a...",False,False,False
2,"@LaCaaautiva Perdón...marido lo ve, no quiero ...","perdon ... marido él ver , no querer que ganar .",False,False,True
3,@manuaguilar3 @alberto_dahik @alberto_dahik no...,no ir a dar el gusto ver dueno el obra const...,False,False,True
4,@x_carbonero @Agronegocios_Uy @uriartec1 A ese...,"a ese ministro no él ver recorrer el granja , ...",False,False,True


,text,lemmad,is_duplicate,no_stem,has_verb
0,Acordei de manhã passando mal e fiquei assim o...,acordar de manha passar mal e ficar assim o di...,False,False,False
1,primeira vez que eu te vi tão bom que foi nós ...,primeiro vez que eu te vir tao bom que ser o d...,False,False,False
2,"Fui vê um vídeo no Instagram do lou tocando, m...","ser ve um video o instagram do lou tocar , me ...",False,False,False
3,@reporterenato Não vi o nome do senador Marcos...,nao vir o nome do senador marco rogerio ... \n...,False,False,False
4,"Eu só queria te contar que eu fui lá fora, e v...","eu so querer te contar que eu ser o ser , e vi...",False,False,False


In [ ]:
b = es_out.loc[~es_bad.loc[:, 'has_verb'], ['text', 'lemmad']]

for i in range(b.shape[0]):
    print(f'CASE:\n')
    print(f'ORIGINAL:\n')
    print(b.iloc[i, 0])
    print(f'LEMMAD:\n')
    print(b.iloc[i, 1])